In [1]:
import pymongo
import re
from collections import OrderedDict
from datetime import datetime

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017", username="", password="")
raw_events_db = myclient["raw_events_db"]
events_collection = raw_events_db["events"]

applications_db = myclient['applications_db']
applications_collection= applications_db['applications']

In [3]:
applications_documents = applications_collection.find({},projection = {"_id":1,"name":1}).sort("name")

app_id_app_name_dict = {}

for x in applications_documents:
    app_id = x['_id'].strip()
    app_name = x['name'].strip()
    app_id_app_name_dict[app_id] = app_name
print("# Applications: ",len(app_id_app_name_dict))

# Applications:  419


In [4]:
#Get events per application
pipeline= [{"$group": {"_id": "$application_id", "count": {"$sum": 1}}}]
app_id_event_count_doc = events_collection.aggregate(pipeline)
app_id_event_count_dict  = {}

total_count = 0
for x in app_id_event_count_doc:
    app_id = x['_id'].strip()
    event_count = x['count']
    total_count += event_count
    app_id_event_count_dict[app_id] = event_count

app_id_event_count_dict =  OrderedDict(sorted(app_id_event_count_dict.items(), key=lambda kv:(kv[1], kv[0]), reverse=True))
for app_id in app_id_event_count_dict:
    print(app_id," - ", app_id_app_name_dict[app_id], " - ", app_id_event_count_dict[app_id])

print("Total Event Count: ",total_count)

37f0228f2734c8010512d447a080fad3  -  stoffice  -  88117
bf57c906fa7d2bb66d07372e41585d96  -  excel  -  73844
cd0fbb7849b9b9d84c4f8e48e7301ecf  -  outlook  -  69798
a2b5ab0a0378213f941106a349e9304b  -  ifs.corp.lpl.com  -  55706
aa25d6af980471b133f7ad0f981aec16  -  vncecmxcp2prd.ncprod.lpl.com  -  23476
f6f4ab435f8857ee0528a815c0f6728c  -  lplecm.crm.dynamics.com  -  8526
0186dcc060cb68888b8a088ec9c9332e  -  msaccess  -  8195
9edc8b8fa912a3d810043b9f17ea7a8a  -  winword  -  217
Total Event Count:  327879


In [12]:
for app_id in app_id_event_count_dict:
    app_id_title_docs = events_collection.find({"application_id": app_id}, projection={'title':1, 'specifications.event_path':1})
    title_set = set({})
    event_path_set = set({})

    for x in app_id_title_docs:
        title_set.add(x['title'])
        event_path_set.add(x['specifications']['event_path'])

    f = open("../Schneider/"+app_id_app_name_dict[app_id].replace(":","_").replace("-","_")+"_title.txt",'w', encoding="utf-8")
    for title in title_set:
        f.write(title)
        f.write("\n")
    f.close()
    f = open("../Schneider/"+app_id_app_name_dict[app_id].replace(":","_").replace("-","_")+"_event_path.txt",'w', encoding="utf-8")
    for event_path in event_path_set:
        f.write(event_path)
        f.write("\n")
    f.close()

In [14]:
input_dir = "../Schneider/"
application_name = 'msaccess'.replace("-","_")


control_type_control_name_set_dict = {}
control_type_automation_id_set_dict = {}
control_type_automation_id_count_dict = {}

f = open(input_dir + application_name+"_event_path.txt",'r', encoding='utf-8')
lines = f.readlines()
f.close()

for line in lines:
    event_path = line.strip()
    if len(event_path) == 0:
        continue
    #if 'user' in event_path.lower():
    #    print(event_path)
    event_path_items = event_path.split("\/")
    for event_path_item in event_path_items:
        event_path_item_elements = event_path_item.split("\|")
        if len(event_path_item_elements) < 4:
            continue
        automation_id = event_path_item_elements[1][3:].strip()
        control_type = event_path_item_elements[2][2:].strip()
        control_name = event_path_item_elements[3][2:].strip()
        
        if control_type in control_type_control_name_set_dict:
            control_type_control_name_set = control_type_control_name_set_dict[control_type]
        else:
            control_type_control_name_set = set({})
        control_type_control_name_set.add(control_name)
        control_type_control_name_set_dict[control_type] = control_type_control_name_set
        
        if control_type in control_type_automation_id_set_dict:
            control_type_automation_id_set = control_type_automation_id_set_dict[control_type]
            
        else:
            control_type_automation_id_set = set({})
        control_type_automation_id_set.add(automation_id)
        control_type_automation_id_set_dict[control_type] = control_type_automation_id_set

        if control_type in control_type_automation_id_count_dict:
            automation_id_count_dict = control_type_automation_id_count_dict[control_type]
        else:
            automation_id_count_dict = {}
        count = 1
        if automation_id in automation_id_count_dict:
            count += automation_id_count_dict[automation_id]
        automation_id_count_dict[automation_id] = count
        control_type_automation_id_count_dict[control_type] = automation_id_count_dict
        
control_type_control_name_count_dict = {}
for control_type in control_type_control_name_set_dict:
    control_type_control_name_count_dict[control_type] = len(control_type_control_name_set_dict[control_type])
control_type_control_name_count_dict = OrderedDict(sorted(control_type_control_name_count_dict.items(), key=lambda kv:(kv[1], kv[0]), reverse=True))

for control_type in control_type_automation_id_count_dict:
    automation_id_count = control_type_automation_id_count_dict[control_type]
    automation_id_count = OrderedDict(sorted(automation_id_count.items(), key=lambda kv:(kv[1], kv[0]), reverse=True))
    control_type_automation_id_count_dict[control_type] = automation_id_count
    
for control_type in control_type_control_name_count_dict:
    print(control_type, control_type_control_name_count_dict[control_type])

50033 122
50029 45
50000 41
50025 35
50032 26
50004 19
50020 12
50034 9
50003 9
50021 6
50011 6
50008 4
50030 2
50026 2
50018 2
50017 2
50014 2
50009 2
50007 2
50037 1
50036 1
50027 1
50006 1
50002 1


In [19]:
# print(control_type_control_name_set_dict)
for control_name in sorted(control_type_control_name_set_dict['50026']):
    # if "Realize Your Potential" in control_name:
        print(control_name)


Tables


In [8]:
automation_id_count_dict = control_type_automation_id_count_dict['50026']
for automation_id in automation_id_count_dict:
    print(automation_id, automation_id_count_dict[automation_id])

mainContent 2986
 538
id-22 147
id-25 139
id-1427 67
id-375 45
id-394 44
id-1594 40
id-1407 34
id-21 32
id-1145 32
id-2745 31
id-1712 30
id-799 29
id-844 28
id-24 27
id-866 26
id-1141 26
id-489 25
id-1097 24
id-580 23
id-16654191-19ad-48ca-9e46-c4b4fbdad76f-17-ownerid-error-message 23
id-1895 22
id-769 21
id-76 21
id-355 21
id-1692 20
id-813 19
id-599 19
id-1159 19
id-554 18
id-1348 18
id-1187 18
id-1776 17
id-1414 17
id-786 16
id-779 16
id-91 15
id-2725 15
id-374 14
id-362 14
id-1328 14
id-1077 14
id-603 13
id-1667 13
id-783 12
id-381 12
id-37 12
id-287 12
id-673 11
id-643 11
id-547 11
id-177 11
id-16780 11
id-1084 11
id-878 10
id-559 10
id-377 10
id-16824 10
id-1581 10
id-1410 10
id-1121 10
Grid95f1f1ee-7eae-d339-95d0-58b3f353f156_component 10
id-9071 9
id-85 9
id-72 9
id-478 9
id-358 9
id-3500 9
id-2416 9
id-1711 9
id-133 9
id-657 8
id-331 8
id-16794 8
id-1398 8
id-853 7
id-831 7
id-671 7
id-551 7
id-541 7
id-476 7
id-469 7
id-391 7
id-3095 7
id-2732 7
id-2728 7
id-271 7
id-2391 7
i

In [9]:
def prune_hashes(sentence):
    regex = re.compile("#[A-Fa-f0-9]{64,}#", re.IGNORECASE)
    modified_sentence = sentence
    matches = re.findall(regex, modified_sentence)
    for match in matches:
        modified_sentence = modified_sentence.replace(match, '')

    return modified_sentence.strip()